In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# -------- Generate Simple Data --------
X1 = np.random.rand(200, 2)   # Input 1
X2 = np.random.rand(200, 1)   # Input 2

y1 = X1[:, 0] + X2[:, 0]      # Output 1
y2 = X1[:, 1]                 # Output 2

# -------- Model Builder --------
def build_model(units):
    i1 = keras.Input(shape=(2,))
    i2 = keras.Input(shape=(1,))

    x1 = layers.Dense(units, activation='relu')(i1)
    x2 = layers.Dense(units, activation='relu')(i2)

    x = layers.concatenate([x1, x2])

    o1 = layers.Dense(1)(x)
    o2 = layers.Dense(1)(x)

    model = keras.Model([i1, i2], [o1, o2])
    model.compile(optimizer='adam', loss='mse')
    return model

# -------- Hyper-Parameter Tuning --------
best_loss = float("inf")

for units in [8, 16]:
    print(f"\nTraining with {units} hidden units")
    model = build_model(units)
    history = model.fit([X1, X2], [y1, y2],
                        epochs=10, batch_size=16, verbose=0)
    loss = history.history['loss'][-1]
    print("Final loss:", loss)

    if loss < best_loss:
        best_loss = loss
        best_model = model

print("\nBest Model Loss:", best_loss)

# -------- Test Prediction --------
pred = best_model.predict([X1[:5], X2[:5]])

print("\nTrue Output:")
print(np.column_stack((y1[:5], y2[:5])))

print("\nPredicted Output:")
print(np.column_stack(pred))


Training with 8 hidden units
Final loss: 0.8464526534080505

Training with 16 hidden units
Final loss: 0.12174487113952637

Best Model Loss: 0.12174487113952637
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step

True Output:
[[0.77576443 0.81216552]
 [1.20014134 0.1393125 ]
 [1.34733826 0.19964421]
 [1.42501386 0.45633508]
 [0.7088419  0.37942484]]

Predicted Output:
[[1.0008904  0.6019724 ]
 [0.8581322  0.3453722 ]
 [1.2246304  0.6187013 ]
 [1.068818   0.5171817 ]
 [0.78789306 0.3887134 ]]


In [ ]:
pip install scikeras==0.13.0 scikit-learn==1.4.2 numpy==1.26.0

  Using cached scikeras-0.13.0-py3-none-any.whl.metadata (3.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.3 MB/s eta 0:00:00
Using cached scikeras-0.13.0-py3-none-any.whl (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 96.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0
  Attempting uninstall: scikeras
    Found existing installation: scikeras 0.4.1
    Uninstalling scikeras-0.4.1:
      Successfully uninstalled scikeras-0.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

In [ ]:
#MIMO
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from scikeras.wrappers import KerasRegressor

def generate_mimo_data(n_samples=1000, n_feat1=5, n_feat2=3):
    np.random.seed(42)
    X1 = np.random.rand(n_samples, n_feat1)
    X2 = np.random.rand(n_samples, n_feat2)
    y1 = 2 * X1[:, 0] + 3 * X2[:, 1] + np.random.randn(n_samples) * 0.1
    y2 = 0.5 * X1[:, 2] - X2[:, 0] + np.random.randn(n_samples) * 0.1
    return X1, X2, np.column_stack((y1, y2))

X1, X2, Y = generate_mimo_data()
X1_train, X1_test, X2_train, X2_test, Y_train, Y_test = train_test_split(
    X1, X2, Y, test_size=0.2, random_state=42
)

scaler_X1, scaler_X2 = StandardScaler(), StandardScaler()
X1_train_scaled = scaler_X1.fit_transform(X1_train)
X1_test_scaled = scaler_X1.transform(X1_test)
X2_train_scaled = scaler_X2.fit_transform(X2_train)
X2_test_scaled = scaler_X2.transform(X2_test)

def create_mimo_model(optimizer='adam', learning_rate=0.01, hidden_units=32):
    input1 = keras.layers.Input(shape=(X1.shape[1],), name='input_1')
    input2 = keras.layers.Input(shape=(X2.shape[1],), name='input_2')
    x1 = keras.layers.Dense(hidden_units, activation='relu')(input1)
    x1 = keras.layers.Dense(hidden_units // 2, activation='relu')(x1)
    x2 = keras.layers.Dense(hidden_units, activation='relu')(input2)
    x2 = keras.layers.Dense(hidden_units // 2, activation='relu')(x2)
    merged = keras.layers.concatenate([x1, x2])
    output1 = keras.layers.Dense(1, name='output_1')(merged)
    output2 = keras.layers.Dense(1, name='output_2')(merged)
    model = keras.Model(inputs=[input1, input2], outputs=[output1, output2])
    opt = keras.optimizers.Adam(learning_rate=learning_rate) if optimizer == 'adam' else keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss={'output_1': 'mse', 'output_2': 'mse'}, metrics=['mae'])
    return model

keras_regressor = KerasRegressor(model=create_mimo_model, verbose=0)

param_grid = {
    'model__optimizer': ['adam', 'sgd'],
    'model__learning_rate': [0.001, 0.01],
    'model__hidden_units': [16, 32, 64],
    'epochs': [10, 20],
    'batch_size': [16, 32]
}

grid_search = GridSearchCV(
    estimator=keras_regressor,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1
)

grid_search.fit([X1_train_scaled, X2_train_scaled], Y_train)

print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best CV score (neg MSE): {grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_.model
loss, mae1, mae2 = best_model.evaluate(
    [X1_test_scaled, X2_test_scaled], Y_test, verbose=0
)

print(f"\nTest Set Performance:")
print(f"  Loss: {loss:.4f}")
print(f"  MAE Output 1: {mae1:.4f}")
print(f"  MAE Output 2: {mae2:.4f}")

preds = best_model.predict([X1_test_scaled[:5], X2_test_scaled[:5]], verbose=0)
print("\nSample Predictions (first 5):")
print(f"  True:      {Y_test[:5]}")
print(f"  Predicted: {np.array(preds).T}")

ValueError: Found input variables with inconsistent numbers of samples: [2, 800]

In [ ]:
pip install scikeras==0.13.0 scikit-learn==1.4.2 numpy==1.26.0
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from scikeras.wrappers import KerasRegressor

def generate_mimo_data(n_samples=1000, n_feat1=5, n_feat2=3):
    np.random.seed(42)
    X1 = np.random.rand(n_samples, n_feat1)
    X2 = np.random.rand(n_samples, n_feat2)
    y1 = 2 * X1[:, 0] + 3 * X2[:, 1] + np.random.randn(n_samples) * 0.1
    y2 = 0.5 * X1[:, 2] - X2[:, 0] + np.random.randn(n_samples) * 0.1
    return X1, X2, np.column_stack((y1, y2))

X1, X2, Y = generate_mimo_data()
X1_train, X1_test, X2_train, X2_test, Y_train, Y_test = train_test_split(
    X1, X2, Y, test_size=0.2, random_state=42
)

scaler_X1, scaler_X2 = StandardScaler(), StandardScaler()
X1_train_scaled = scaler_X1.fit_transform(X1_train)
X1_test_scaled = scaler_X1.transform(X1_test)
X2_train_scaled = scaler_X2.fit_transform(X2_train)
X2_test_scaled = scaler_X2.transform(X2_test)

X_train_combined = np.hstack((X1_train_scaled, X2_train_scaled))
X_test_combined = np.hstack((X1_test_scaled, X2_test_scaled))

def create_mimo_model(optimizer='adam', learning_rate=0.01, hidden_units=32, input_dim=None):
    if input_dim is None:
        input_dim = X_train_combined.shape[1]
    inp = keras.layers.Input(shape=(input_dim,), name='input_combined')
    x = keras.layers.Dense(hidden_units, activation='relu')(inp)
    x = keras.layers.Dense(hidden_units // 2, activation='relu')(x)
    # single output with 2 units (for the two targets) — makes scikeras/GridSearch compatible
    outputs = keras.layers.Dense(2, activation='linear', name='outputs')(x)
    model = keras.Model(inputs=inp, outputs=outputs)
    opt = keras.optimizers.Adam(learning_rate=learning_rate) if optimizer == 'adam' else keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='mse', metrics=['mae'])
    return model

keras_regressor = KerasRegressor(model=create_mimo_model, verbose=0)

param_grid = {
    'model__optimizer': ['adam', 'sgd'],
    'model__learning_rate': [0.001, 0.01],
    'model__hidden_units': [16, 32, 64],
    'epochs': [10, 20],
    'batch_size': [16, 32]
}

grid_search = GridSearchCV(
    estimator=keras_regressor,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1
)

grid_search.fit(X_train_combined, Y_train)

print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best CV score (neg MSE): {grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_.model_  # note the trailing underscore
eval_results = best_model.evaluate(X_test_combined, Y_test, verbose=0)
print(f"\nRaw evaluate() output: {eval_results}")

# evaluate() returns [loss, mae] for this single-output (2-unit) configuration
loss = eval_results[0]
mae = eval_results[1]

print(f"\nTest Set Performance:")
print(f"  Loss: {loss:.4f}")
print(f"  MAE (averaged over outputs): {mae:.4f}")

preds = best_model.predict(X_test_combined[:5], verbose=0)
print("\nSample Predictions (first 5):")
print(f"  True:      {Y_test[:5]}")
print(f"  Predicted: {preds}")


Fitting 3 folds for each of 48 candidates, totalling 144 fits
